In [1]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a photography of a woman and her dog on the beach


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


woman sitting on the beach with her dog and a cell phone


In [5]:
import pandas as pd

In [6]:
df=pd.read_csv('/content/song_data.csv')

In [8]:
df.columns

Index(['Unnamed: 0', 'Song', 'Album', 'Album Release Date', 'Artist Names',
       'Artist(s) Genres', 'Hot100 Ranking Year', 'Hot100 Rank',
       'Song Length(ms)', 'Spotify Link', 'Song Image', 'Spotify URI',
       'Popularity', 'Acousticness', 'Danceability', 'Energy',
       'Instrumentalness', 'Liveness', 'Loudness', 'Speechiness', 'Tempo',
       'Valence', 'Key', 'Mode', 'Time Signature',
       'number of words of the song name',
       'number of characters of the song name'],
      dtype='object')

In [9]:
urls=df['Song Image'].tolist()

In [11]:
from tqdm import tqdm

In [12]:
output_descriptions=[]
for url in tqdm(urls, desc="Processing images", unit="image"):

    raw_image = Image.open(requests.get(url, stream=True).raw).convert('RGB')


    inputs = processor(raw_image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs)
    desc = processor.decode(out[0], skip_special_tokens=True)


    output_descriptions.append(desc)

Processing images:   0%|          | 0/6191 [00:00<?, ?image/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Processing images: 100%|██████████| 6191/6191 [51:08<00:00,  2.02image/s]


In [14]:
import csv

csv_file_path = 'images_desc.csv'


# Open CSV file in write mode with UTF-8 encoding
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)  # Create a CSV writer
    writer.writerow(['image_description'])  # Write column header
    for value in output_descriptions:
        writer.writerow([value])
